<a href="https://colab.research.google.com/github/hugotomita1201/yachay.ai_project/blob/main/electra_grouped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece
!pip install transformers


# %%
import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from transformers import BertConfig
import pandas as pd
import csv
import time
import pickle
from torch.optim import AdamW, Adam
from sklearn.metrics import accuracy_score
import numpy as np


import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# use gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df = pd.read_csv('df_10_grouped_new_text.csv')

# Load pre-trained model tokenizer (vocabulary)
model_name = 'google/electra-small-discriminator'

config = BertConfig.from_pretrained(model_name)
config.num_labels = 10  # number of regions for classification
bert_model = AutoModel.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(
    model_name)
#config.max_length = 512

#bert_model = RobertaModel.from_pretrained(model_name, config=config)
#tokenizer = BertweetTokenizer.from_pretrained(model_name)

# preprocess data with function

'''
weights = [0.1668145519077196,
 0.11091393078970718,
 0.04214729370008873,
 0.1774622892635315,
 0.09316770186335403,
 0.2440106477373558,
 0.11091393078970718,
 0.006211180124223602,
 0.006211180124223602,
 0.04214729370008873]
'''

#weights = torch.tensor(weights).float().to(device)

def haversine_distance(lat1, lon1, lat2, lon2):
    earth_radius = 6371  # Earth radius in km
    pi = torch.tensor(3.141592653589793, dtype=torch.float)

    lat1, lon1, lat2, lon2 = [torch.tensor(
        x.to_numpy(), dtype=torch.float) * (pi / 180) for x in [lat1, lon1, lat2, lon2]]

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = torch.sin(dlat / 2) ** 2 + torch.cos(lat1) * \
        torch.cos(lat2) * torch.sin(dlon / 2) ** 2
    c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1 - a))

    distance = earth_radius * c
    return distance


class HaversineLoss(nn.Module):
    def __init__(self):
        super(HaversineLoss, self).__init__()

    def forward(self, y_pred, y_true):
        lat1, lon1 = y_pred[:, 0], y_pred[:, 1]
        lat2, lon2 = y_true[:, 0], y_true[:, 1]

        distance = haversine_distance(lat1, lon1, lat2, lon2)
        loss = distance.mean()
        return loss


df['text_combined'] = '" ' + df['text'] + ' ' + df['text_3'] + ' "'

def preprocess_data(df, tokenizer):


    # Filter and sample data
    # Split into train and test sets
    train_features, test_features, train_labels, test_labels = train_test_split(
        df['text_combined'], df[['group']], test_size=0.2, random_state=12345)
    train_featuresx, test_featuresx, train_labelsx, test_labels_centroids = train_test_split(
        df['group'], df[['group_centroid_lat', 'group_centroid_lng']], test_size=0.2, random_state=12345)


    # Tokenize the training and test text features
    train_encodings = tokenizer(train_features.values.tolist(
    ), padding=True, truncation=True, return_tensors='pt', max_length=256)
    test_encodings = tokenizer(test_features.values.tolist(
    ), padding=True, truncation=True, return_tensors='pt', max_length=256)

    # Tokenize the training and test text features
    train_input_ids = train_encodings['input_ids'].to(device)
    test_input_ids = test_encodings['input_ids'].to(device)

    # make attention masks to let know which tokens are real and which are padding
    train_attention_mask = train_encodings['attention_mask'].to(device)
    test_attention_mask = test_encodings['attention_mask'].to(device)

    # Convert labels to PyTorch tensors with long datatype
    train_labels = torch.tensor(
        train_labels['group'].values, dtype=torch.long).to(device)
    test_labels = torch.tensor(
        test_labels['group'].values, dtype=torch.long).to(device)

    return train_encodings, test_encodings, train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels, test_labels_centroids


start_time = time.time()
# call on function
train_encodings, test_encodings, train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels, test_labels_centroids = preprocess_data(
    df, tokenizer)
end_time = time.time()
print(f"Preprocessing took {end_time - start_time} seconds")
# print out the shape of the data

print(train_input_ids.shape)
print(test_input_ids.shape)

# Define the BERT-based classification model


class BERTClassifier(nn.Module):
    def __init__(self, bert_model, num_regions):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.10)
        self.classifier = nn.Linear(config.hidden_size, num_regions)

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(
            input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_outputs[0][:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits


num_regions = 10

# Initialize the BERT-based classification model
bert_classifier = BERTClassifier(bert_model, num_regions).to(device)

# Define the optimizer and the loss function
optimizer = AdamW(bert_classifier.parameters(), lr=5e-6, weight_decay = 1e-9)
loss_fn = nn.CrossEntropyLoss()

# %%
# Train the model
num_epochs = 5
batch_size = 16

# set a timer to see how long it takes to train the model

start_time = time.time()
# print starting time
print(f"Training started at {time.ctime(start_time)}")
for epoch in range(num_epochs):
    for i in range(0, len(train_input_ids), batch_size):
        input_ids_batch = train_input_ids[i:i+batch_size].to(device)
        attention_mask_batch = train_attention_mask[i:i+batch_size].to(device)
        region_ids_batch = train_labels[i:i+batch_size].to(device)

        bert_classifier.zero_grad()

        logits = bert_classifier(
            input_ids_batch, attention_mask_batch).to(device)
        loss = loss_fn(logits, region_ids_batch)

        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set after each epoch
    test_batch_size = 8

    # Evaluate the model on the test set after each epoch
    with torch.no_grad():
        test_losses = []
        test_accuracies = []
        test_distances = []
        test_predictions = []
        test_actual_labels = []

        for i in range(0, len(test_input_ids), test_batch_size):
            test_input_ids_batch = test_input_ids[i:i +
                                                  test_batch_size].to(device)
            test_attention_mask_batch = test_attention_mask[i:i+test_batch_size].to(
                device)
            test_labels_batch = test_labels[i:i+test_batch_size].to(device)

            test_logits_batch = bert_classifier(
                test_input_ids_batch, test_attention_mask_batch)
            test_loss_batch = loss_fn(test_logits_batch, test_labels_batch)

            test_losses.append(test_loss_batch.item())

            # append test predictions and labels to list for each epoch
            test_predictions.append(torch.argmax(
                test_logits_batch, dim=1).cpu().numpy())
            test_actual_labels.append(test_labels_batch.cpu().numpy())

        # make arrays into dataframes to merge with df
        test_predictions_df = pd.DataFrame(np.concatenate(
            test_predictions, axis=0).reshape(-1, 1), columns=['group'])
        test_actual_labels_df = pd.DataFrame(np.concatenate(
            test_actual_labels, axis=0).reshape(-1, 1), columns=['group'])

        # calculate centroids for predicted and labels groups
        predicted_centroids = test_predictions_df.merge(df[['group', 'group_centroid_lat', 'group_centroid_lng']].drop_duplicates(
            subset=['group']), on='group', how='left')[['group_centroid_lat', 'group_centroid_lng']]
        labels_centroids = test_actual_labels_df.merge(df[['group', 'group_centroid_lat', 'group_centroid_lng']].drop_duplicates(
            subset=['group']), on='group', how='left')[['group_centroid_lat', 'group_centroid_lng']]

        # calculate haversine distance based on predicted and labels centroids
        haversine_distances = haversine_distance(predicted_centroids['group_centroid_lat'], predicted_centroids[
                                                 'group_centroid_lng'], labels_centroids['group_centroid_lat'], labels_centroids['group_centroid_lng'])

        avg_distance = sum(haversine_distances) / len(haversine_distances)

        avg_test_loss = sum(test_losses) / len(test_losses)
        test_accuracy = accuracy_score(
            test_actual_labels_df, test_predictions_df)
        print(
            f"Epoch {epoch+1}: Test loss={avg_test_loss:.4f}, Test accuracy={test_accuracy:.4f}, Avg distance={avg_distance:.4f} km")


end_time = time.time()
print(f"Training took {end_time - start_time} seconds")


# %%
torch.save(bert_classifier.state_dict(),
           'bert_classifier_for_grouped_data_2.pth')

'''

}
'''
# %%


#save the predictions to a csv file
test_predictions_df.to_csv('test_predictions_2.csv', index=False)

#save the test actual labels to a csv file
test_actual_labels_df.to_csv('test_actual_labels_2.csv', index=False)

#save 
# %%



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Preprocessing took 75.4535641670227 seconds
torch.Size([282567, 256])
torch.Size([70642, 256])
Training started at Sun May 28 01:57:09 2023
Epoch 1: Test loss=1.7096, Test accuracy=0.3436, Avg distance=1352.7168 km
Epoch 2: Test loss=1.6822, Test accuracy=0.3546, Avg distance=1332.9055 km
Epoch 3: Test loss=1.6752, Test accuracy=0.3556, Avg distance=1330.7740 km
Epoch 4: Test loss=1.6848, Test accuracy=0.3544, Avg distance=1330.5020 km


In [ ]:
from google.colab import files

files.download('bert_classifier_for_grouped_data_2.pth')

#download the test predictions to local machine
files.download('test_predictions_2.csv')

#download the test actual labels to local machine
files.download('test_actual_labels_2.csv')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored